# 6893-Final Project
### -- Google Analytics Customer Revenue Prediction 
[dataset](https://www.kaggle.com/c/ga-customer-revenue-prediction/data)

## Part 1 : Data Cleaning
### 1) Sample Data

Since our data is quite large (23.67GB), in order to explore our data more efficiently, we will sample data of 25,000 unique customers (using column 'fullVisitorId') for EDA.

In [4]:
import pandas as pd
from pandas import Series, DataFrame
import glob

Our data looks like:

In [66]:
show = pd.read_csv('/home/xc2418/data/train_v2.csv', nrows = 2)
show

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307


Resulting from our big data, we will split our training data into 9 pieces ('split -l 200000 data.csv'), sample them seperately, and concat them.

In [3]:
base_data = pd.read_csv('/home/xc2418/data/splited/base_data', dtype = {'fullVisitorId': 'str'})

In [6]:
# sample n = 10000 unique customers 
import random as rd
rd.seed(1)

n = 25000
id_unique = base_data['fullVisitorId'].unique()
id_unique_l = list(id_unique)
index = rd.sample(id_unique_l, n)
index = [str(i) for i in index]

In [7]:
# sample from base_data
base_data['fullVisitorId'] = base_data['fullVisitorId'].astype('str')
data_samp = base_data.loc[base_data['fullVisitorId'].isin(index)]

In [8]:
# sample from other pieces
df = data_samp
for path in glob.glob('/home/xc2418/data/splited/x*'):
    # read splited files
    df_temp = pd.read_csv(path, header = None, dtype = {'fullVisitorId': 'str'})
    # rename the columns
    df_temp.columns = data_samp.columns
    # sample
    df_temp['fullVisitorId'] = df_temp['fullVisitorId'].astype('str')
    df_samp = df_temp.loc[df_temp['fullVisitorId'].isin(index)]
    # append the sampled dataframe to the previous dataframe
    df = pd.concat([df, df_samp], ignore_index = True)

/home/xc2418/ENTER/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Now we get the sampled data of 25,000 unique customers.

In [11]:
# write sampled data
df.to_csv("/home/xc2418/data/sampled_data.csv", header = True, index = False)  

### 2) Flatten data

Also, we can see that there are multiple columns containing JSON blobs of varying depth. Then we are going to flatten these JSON columns.

In [2]:
import numpy as np 
import json
import pandas.io.json as pdjson
import ast
import os

In [3]:
def flatten(path):

    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(path, 
                     converters = {column: json.loads for column in JSON_COLUMNS}, 
                     dtype = {'fullVisitorId': 'str'})
       
    df['hits'] = df['hits'].apply(ast.literal_eval)
    df['hits'] = df['hits'].str[0]
    df['hits'] = df['hits'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)
    
    df['customDimensions'] = df['customDimensions'].apply(ast.literal_eval)
    df['customDimensions'] = df['customDimensions'].str[0]
    df['customDimensions'] = df['customDimensions'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)
    
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource','hits','customDimensions']

    for column in JSON_COLUMNS:
        column_as_df = pdjson.json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis = 1).merge(column_as_df, right_index = True, left_index = True)
    
    return df

In [7]:
df_flat = flatten('/home/xc2418/data/sampled_data.csv')

There are still some columns in JSON format: (they are in form of list)

In [8]:
json_list = []

# for each column
for i in range(len(df_flat.columns)): 
    # see if some element 1 is a list
    if (isinstance(df_flat.iloc[1,i], list)): 
        # save the list name to json_list
        json_list.append(df_flat.columns[i])   
        
print(json_list)

['hits_customDimensions', 'hits_customMetrics', 'hits_customVariables', 'hits_experiment', 'hits_product', 'hits_promotion', 'hits_publisher_infos']


### 3) Reduce Meaningless Features
#### (1) Reduce Meaningless JSON Columns 

Let's look at how many unique values the remaining JSON columns have:

In [9]:
for col in json_list:
    coll = df_flat[col].astype('str')
    n = coll.nunique(dropna = False)
    print('%s has %d unique values.' %(col, n))

hits_customDimensions has 2 unique values.
hits_customMetrics has 2 unique values.
hits_customVariables has 2 unique values.
hits_experiment has 2 unique values.
hits_product has 2388 unique values.
hits_promotion has 17 unique values.
hits_publisher_infos has 2 unique values.


Then we take a look at what do their unique values look like:

In [10]:
for col in json_list:
    coll = df_flat[col].astype('str')
    n = coll.nunique(dropna = False)
    if n == 2:
        print('### %s unique values: %s' %(col, coll.unique()))
    else:
        print('### %s looks like: %s' %(col, coll.unique()[1]))

### hits_customDimensions unique values: ['[]' 'nan']
### hits_customMetrics unique values: ['[]' 'nan']
### hits_customVariables unique values: ['[]' 'nan']
### hits_experiment unique values: ['[]' 'nan']
### hits_product looks like: [{'productSKU': 'GGOEYFKQ020699', 'v2ProductName': 'YouTube Custom Decals', 'v2ProductCategory': 'Home/Shop by Brand/YouTube/', 'productVariant': '(not set)', 'productBrand': '(not set)', 'productPrice': '1990000', 'localProductPrice': '1990000', 'isImpression': True, 'customDimensions': [], 'customMetrics': [], 'productListName': 'Category', 'productListPosition': '1'}, {'productSKU': 'GGOEYDHJ056099', 'v2ProductName': '22 oz YouTube Bottle Infuser', 'v2ProductCategory': 'Home/Shop by Brand/YouTube/', 'productVariant': '(not set)', 'productBrand': '(not set)', 'productPrice': '4990000', 'localProductPrice': '4990000', 'isImpression': True, 'customDimensions': [], 'customMetrics': [], 'productListName': 'Category', 'productListPosition': '2'}, {'productSK

We can see that the 5 JSON columns which has 2 unique values ('[  ]' and 'nan') don't provide any useful information, so we can drop them. <br>
As for 'hits_product' and 'hits_promotion', they may provide few contributions to our result compared to the efforts we'll take to deal with them. So we will drop them.

In [11]:
for col in json_list:
    del(df_flat[col])

In [12]:
print('Now our flattened dataset has %s columns.' %df_flat.columns.shape)

Now our flattened dataset has 121 columns.


#### (2) Reduce Constant Columns

If a column has only 1 unique value, this column won't contribute to our final result, so we can drop this kind of column.

In [13]:
value_1 = []

for col in df_flat.columns:
    try:
        n = df_flat[col].nunique(dropna = False) # including NAs
        if n == 1: 
            value_1.append(col)
            # if this column only contains 1 unique value, we delete it
            del(df_flat[col])
            
    except TypeError:
        coll = df_flat[col].astype('str')
        n = coll.nunique(dropna = False)
        if n == 1:
            value_1.append(col)
            del(df_flat[col])

In [14]:
print('The columns we have just deleted are %s, and our dataframe has %d columns now.' %(value_1, df_flat.columns.size))

The columns we have just deleted are ['socialEngagementType', 'device_browserSize', 'device_browserVersion', 'device_flashVersion', 'device_language', 'device_mobileDeviceBranding', 'device_mobileDeviceInfo', 'device_mobileDeviceMarketingName', 'device_mobileDeviceModel', 'device_mobileInputSelector', 'device_operatingSystemVersion', 'device_screenColors', 'device_screenResolution', 'geoNetwork_cityId', 'geoNetwork_latitude', 'geoNetwork_longitude', 'geoNetwork_networkLocation', 'totals_visits', 'trafficSource_adwordsClickInfo.criteriaParameters', 'hits_index', 'hits_value'], and our dataframe has 100 columns now.


#### (3) Reduce Almost Constant Columns

For some features, one single category may dominant (accounts for 95% or more). Then this feature is almost constant, and will contribute few to our prediction, so we'll drop this kind of columns.

In [39]:
# define a function to find the max component for each column
n = df_flat.shape[0]
def max_freq(col, df = df_flat, n = n):
    count = df[col].value_counts()
    freq = count/n
    return(max(freq))

In [42]:
# drop the column if the largest propotion is larger than 90%
# print the columns we dropped
for col in df_flat.columns:
    if max_freq(col) > 0.9:
        print(col,':',max_freq(col))
        del(df_flat[col])     

trafficSource_campaign : 0.9420594005200967
hits_appInfo.screenDepth : 0.9992928509512295
hits_contentGroup.contentGroup1 : 0.9953465030338975
hits_contentGroup.contentGroup3 : 0.9780099457092021
hits_contentGroup.contentGroup4 : 0.9992928509512295
hits_contentGroup.contentGroup5 : 0.9992928509512295
hits_contentGroup.previousContentGroup1 : 0.9992928509512295
hits_contentGroup.previousContentGroup2 : 0.9992928509512295
hits_contentGroup.previousContentGroup3 : 0.9992928509512295
hits_contentGroup.previousContentGroup4 : 0.9992928509512295
hits_contentGroup.previousContentGroup5 : 0.9992928509512295
hits_eCommerceAction.action_type : 0.9940462612345454
hits_eCommerceAction.step : 0.9992016059126785
hits_exceptionInfo.isFatal : 0.9992928509512295
hits_hitNumber : 0.99023678087504
hits_isEntrance : 0.9956886719284639
hits_isInteraction : 0.9992928509512295
hits_page.pagePathLevel4 : 0.9018659610383686
hits_social.socialInteractionNetworkAction : 0.9992928509512295
hits_time : 0.999292850

In [43]:
print('Now our dataset has %s columns' %df_flat.shape[1])

Now our dataset has 79 columns


#### (4) Drop Deprecated Fields
According to the [Description of Dataset by Google](https://support.google.com/analytics/answer/3437719?hl=en), we will drop 2 deprecated fields: ['totals_totalTransactionRevenue', 'device_isMobile'].

In [4]:
del[df_flat['totals_totalTransactionRevenue']]
del[df_flat['device_isMobile']]

In [6]:
# write cleaned data
df_flat.to_csv("/home/xc2418/data/sampled_data_cleaned.csv", header = True, index = False)  

### 4) Flatten and Clean the Whole Data

In [4]:
# name splited files except for the first one(has column names)
col_name = pd.read_csv('/home/xc2418/final_proj/data/splited/xaa').columns

path = '/home/xc2418/final_proj/data/splited'
filenames = glob.glob(path + '/x*')

for file in filenames:
    spacename = file.split("/")[-1]
    # read splited files
    data = pd.read_csv(file, header = None)
    # rename the columns
    data.columns = col_name
    
    data.to_csv('/home/xc2418/final_proj/data/splited/named/%s.csv' %spacename, 
                header = True, index = False)

In [6]:
col = ['hits_customDimensions', 'hits_customMetrics', 'hits_customVariables', 
       'hits_experiment', 'hits_product', 'hits_promotion', 'hits_publisher_infos',
       'socialEngagementType', 'device_browserSize', 'device_browserVersion', 
       'device_flashVersion', 'device_language', 'device_mobileDeviceBranding', 
       'device_mobileDeviceInfo', 'device_mobileDeviceMarketingName', 'device_mobileDeviceModel', 
       'device_mobileInputSelector', 'device_operatingSystemVersion', 'device_screenColors', 
       'device_screenResolution', 'geoNetwork_cityId', 'geoNetwork_latitude', 'geoNetwork_longitude',
       'geoNetwork_networkLocation', 'totals_visits', 'trafficSource_adwordsClickInfo.criteriaParameters', 
       'hits_index', 'hits_value', 'trafficSource_campaign', 'hits_appInfo.screenDepth', 
       'hits_contentGroup.contentGroup1', 'hits_contentGroup.contentGroup3', 'hits_contentGroup.contentGroup4',
       'hits_contentGroup.contentGroup5', 'hits_contentGroup.previousContentGroup1', 'hits_contentGroup.previousContentGroup2', 
       'hits_contentGroup.previousContentGroup3', 'hits_contentGroup.previousContentGroup4',
       'hits_contentGroup.previousContentGroup5', 'hits_eCommerceAction.action_type',
       'hits_eCommerceAction.step', 'hits_exceptionInfo.isFatal', 'hits_hitNumber', 'hits_isEntrance',
       'hits_isInteraction', 'hits_page.pagePathLevel4', 'hits_social.socialInteractionNetworkAction', 
       'hits_time hits_type', 'totals_totalTransactionRevenue', 'device_isMobile', 'hits_minute'] 

In [6]:
# flatten splited files
path = '/home/xc2418/final_proj/data/splited/named'
filenames = glob.glob(path + '/*.csv')

for file in filenames:
    spacename = file.split("/")[-1]
    data = flatten(file)
    # delete usless columns
    for coll in col:
        del(data[coll])
    data.to_csv('/home/xc2418/final_proj/data/flattened/%s' %spacename,
                header = True, index = False)

In [7]:
# merge all the files
for file in glob.glob('/home/xc2418/final_proj/data/flattened/*'):
    # skip the headers
    data = pd.read_csv(file, skiprows = [0]) 
    data.to_csv('/home/xc2418/final_proj/data/flat_data.csv', mode = 'a+', index = False)